In [315]:
import numpy as np

In [316]:
BERTH = 10
numIteration = 1000

radio_REVENUE = 1.2 # > 0
radio_ARRIVAL = 0.5 # (0, 1)

REVENUE_T = 0.1
REVENUE_C = radio_REVENUE * REVENUE_T

p_C = radio_ARRIVAL * 0.52
p_T = 0.52 - p_C
p_deC = p_C - 0.02
p_deT = p_T - 0.02

In [317]:
def discharge(y):
  z = REVENUE_C - (REVENUE_C - REVENUE_T) * y / BERTH
  return z

In [318]:
def HC(x, y, V, discount_factor=1.0):
#     A = np.zeros(3)
    A = np.full(3, -np.inf)
    if y > 0 and x + y < BERTH:
        A[0] = discount_factor * V[x+1][y] + REVENUE_C
        A[1] = discount_factor * V[x+1][y-1] + REVENUE_C - discharge(y)
        A[2] = discount_factor * V[x][y]
    elif y == 0 and x < BERTH:
        A[0] = discount_factor * V[x+1][y] + REVENUE_C
        A[2] = discount_factor * V[x][y]
    elif y > 0 and x + y == BERTH:
        A[1] = discount_factor * V[x+1][y-1] + REVENUE_C - discharge(y)
        A[2] = discount_factor * V[x][y]
    elif y == 0 and x == BERTH:
        A[2] = discount_factor * V[x][y]
    else:
        print('Wrong number')
    return  np.argmax(A) + 1, np.max(A)
    
def HT(x, y, V, discount_factor=1.0):
#     A = np.zeros(2)
    A = np.full(2, -np.inf)
    if x + y < BERTH:
        A[0] = discount_factor * V[x][y+1] + REVENUE_T
        A[1] = discount_factor * V[x][y]
    elif x + y == BERTH:
         A[1] = discount_factor * V[x][y]
    else:
        print('Wrong number')
    return  np.argmax(A) + 1, np.max(A)

In [319]:
def value_iteration(p_c, p_t, p_dec, p_det, max_iteration=-1, theta=0.03, discount_factor=1.0):
            
    v = np.zeros((BERTH + 1, BERTH + 1))
    policy_C = np.zeros((BERTH + 1, BERTH + 1))
    policy_T = np.zeros((BERTH + 1, BERTH + 1))
    
    count = 0
    while True:
        count += 1
        delta = 0.
        value = np.zeros((BERTH + 1, BERTH + 1))
        for x in range(BERTH + 1):
            for y in range(BERTH - x + 1):
                _, best_value_c = HC(x, y, v)
                _, best_value_t = HT(x, y, v)
                value[x][y] = p_c * best_value_c + p_t * best_value_t
                if x == 0 and y == 0:
                    value[x][y] += (1. - p_c - p_t) * v[x][y]
                elif x > 0 and y == 0:
                    value[x][y] += p_dec * (x / BERTH) * v[x-1][y] + (1. - p_c - p_t - p_dec * (x / BERTH)) * v[x][y]
                elif x == 0 and y > 0:
                    value[x][y] += p_det * (y / BERTH) * v[x][y-1] + (1. - p_c - p_t - p_det * (y / BERTH)) * v[x][y]
                else:
                    value[x][y] += p_dec * (x / BERTH) * v[x-1][y] + p_det * (y / BERTH) * v[x][y-1] +\
                                  (1 - p_c - p_t - p_dec * (x / BERTH) - p_det * (y / BERTH)) * v[x][y]
                    
                delta = max(delta, np.abs(value[x][y] - v[x][y]))
                v[x][y] = value[x][y]
                
#         print(delta)       
        if delta < theta:
            break
        if max_iteration > 0 and count >= max_iteration:
            break
            
    for x in range(BERTH + 1):
        for y in range(BERTH - x + 1):
            policy_C[x][y], _ = HC(x, y, v)
            policy_T[x][y], _ = HT(x, y, v)
            
    return v, policy_C, policy_T

In [320]:
Value, Action_C, Action_T = value_iteration(p_C, p_T, p_deC, p_deT, numIteration)
print(Value)
print(Action_C)
print(Action_T)

[[32.89047992 32.84479453 32.79570684 32.74292117 32.68610308 32.62487282
  32.55879735 32.48738047 32.41005064 32.32614598 32.2348955 ]
 [32.84004957 32.79057614 32.73736798 32.68008629 32.61834571 32.55170631
  32.47966352 32.4016357  32.31694867 32.22481632  0.        ]
 [32.78577892 32.73215055 32.6744061  32.61215548 32.54495222 32.47228355
  32.39355775 32.30808834 32.21507413  0.          0.        ]
 [32.72732592 32.66912303 32.60636385 32.53859717 32.46530274 32.385879
  32.29962722 32.20573114  0.          0.          0.        ]
 [32.66430641 32.60104722 32.5327199  32.45880038 32.37867885 32.29164473
  32.1968668   0.          0.          0.          0.        ]
 [32.59628823 32.5274177  32.45287845 32.37206016 32.2842439  32.18858414
   0.          0.          0.          0.          0.        ]
 [32.5227839  32.44766174 32.36615745 32.27756052 32.18101909  0.
   0.          0.          0.          0.          0.        ]
 [32.4432409  32.36113085 32.27177583 32.1743544   

In [321]:
# w = np.zeros((BERTH+1,BERTH+1))
# u = np.zeros((BERTH+1,BERTH+1))
# temp_min = 9999999
# for i in range(0, BERTH+1):
#   for j in range(0, BERTH+1):
#     if (Value[i, j] > 0) & (temp_min > Value[i, j]):        
#       temp_min = Value[i, j]

# for i in range(0, BERTH+1):
#   for j in range(0, BERTH+1):
#     if (Value[i, j] > 0):
#       u[i, j] = Value[i, j] - temp_min
#       w[i, j] = (Value[i, j] - temp_min) / (Value.max() - temp_min)
        
# print(u)
# print(w)